In [33]:
import numpy as np
import pandas as pd
import math
import re
import sys
from collections import Counter, OrderedDict, namedtuple, defaultdict
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache

In [13]:
sys.setrecursionlimit(1500)

In [2]:
with open("16-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'\\...\\..........|..\\........../...............................\\.\\.....................................-........\n..........................--............/.....-.../............-........................\\./.\\|\\...............\n...|.-\\.|.......|......../.....-........|........................\\......|..............\\......|...............\n...-\\.....................\\......../....../..............................|........-.........\\...-\\....-.......\n.........\\|............/-....-......-................\\....-........|./.........-..|....................|......\n..-.....\\........................-.....-..../..............||.............-.................|..........-.-././\n......................-\\......................................-/......||.........-.-......./..................\n|..\\................|.....\\.............\\.......|........-..................\\|.................|...-..........\n............................|.....|.......\\../..........\\......\\../..|..../..

In [6]:
test_data_raw = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|...."""



def preprocess_data (data):
    return np.array([list(row)  for row in data.split("\n")], dtype='U10')

test_data = preprocess_data(test_data_raw)
test_data

array([['.', '|', '.', '.', '.', '\\', '.', '.', '.', '.'],
       ['|', '.', '-', '.', '\\', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '|', '-', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '|', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '\\'],
       ['.', '.', '.', '.', '/', '.', '\\', '\\', '.', '.'],
       ['.', '-', '.', '-', '/', '.', '.', '|', '.', '.'],
       ['.', '|', '.', '.', '.', '.', '-', '|', '.', '\\'],
       ['.', '.', '/', '/', '.', '|', '.', '.', '.', '.']], dtype='<U10')

In [7]:
data = preprocess_data(data_raw)
data

array([['\\', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U10')

In [39]:
Point = namedtuple("Point", ["x","y"])

def solution (data, verbose=False):


    def is_in_field(point):
        if (point.x >= 0) and (point.x < data.shape[0]) and (point.y >= 0) and (point.y < data.shape[1]):
            return True
        else:
            return False  
    
    def step_in_direction(point, direction):
        if direction == "U":
            return Point(point.x-1, point.y)
        elif direction == "D":
            return Point(point.x+1, point.y)
        elif direction == "R":
            return Point(point.x, point.y+1)
        elif direction == "L":
            return Point(point.x, point.y-1)

    def get_debug_board (data, point, direction):
        current_position = np.zeros_like(data)
        
        if direction == "U":
            direction_symbol = "^"
        elif direction == "D":
            direction_symbol = "v"
        elif direction == "R":
            direction_symbol = ">"
        elif direction == "L":
            direction_symbol = "<"
        current_position[point] = direction_symbol
        return current_position

    light_cache = defaultdict(list)

    def light_ray(point, direction, step, data, record):
        maxstep = 10000
        next_point = point
        current_direction = direction
        
            
        while step <= maxstep:
            symbol = data[next_point]
            record[next_point] = "#"
            if (light_cache[next_point]) and (current_direction in light_cache[next_point]):
                break
            else:
                light_cache[next_point] = light_cache[next_point] + [current_direction]

            if verbose:
                debug_board = get_debug_board(data, next_point, current_direction)
                print(debug_board)
            if symbol == ".":
                next_point = step_in_direction(next_point, current_direction)
                if not is_in_field(next_point):
                    break
            elif symbol == "/":
                if current_direction == "U":
                    current_direction = "R"
                elif current_direction == "D":
                    current_direction = "L"
                elif current_direction == "R":
                    current_direction = "U"
                elif current_direction == "L":
                    current_direction = "D"
                
                next_point = step_in_direction(next_point, current_direction)

            elif symbol == "\\":
                if current_direction == "U":
                    current_direction = "L"
                elif current_direction == "D":
                    current_direction = "R"
                elif current_direction == "R":
                    current_direction = "D"
                elif current_direction == "L":
                    current_direction = "U"
                
                next_point = step_in_direction(next_point, current_direction)
            

            elif ((symbol == "|") and (current_direction in ["U", "D"])) or ((symbol == "-") and (current_direction in ["L", "R"])) :
                next_point = step_in_direction(next_point, current_direction)

            elif (symbol == "|") and (current_direction in ["L", "R"]):
                current_direction = "U"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, step+1, data, record)
                current_direction = "D"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, step+1, data, record)
            elif (symbol == "-") and (current_direction in ["U", "D"]):
                current_direction = "L"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, step+1, data, record)
                current_direction = "R"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, step+1, data, record)
            if not is_in_field(next_point):
                break
            else:
                step += 1
                if verbose:
                    print(next_point, current_direction)


    record = np.zeros_like(data)
    light_ray(Point(0,0), "R", 1, data, record)

    return record


sol = solution(data)
# sol = solution(test_data, verbose=False)

print(sol)
print(np.count_nonzero(sol))
# display(sum(sol))


[['#' '' '' ... '' '' '']
 ['#' '' '' ... '' '' '']
 ['#' '' '' ... '' '' '']
 ...
 ['' '' '' ... '' '' '']
 ['' '' '' ... '' '' '']
 ['' '' '' ... '' '' '']]
6906


# Part 2

In [41]:
Point = namedtuple("Point", ["x","y"])

def solution (data, verbose=False):


    def is_in_field(point):
        if (point.x >= 0) and (point.x < data.shape[0]) and (point.y >= 0) and (point.y < data.shape[1]):
            return True
        else:
            return False  
    
    def step_in_direction(point, direction):
        if direction == "U":
            return Point(point.x-1, point.y)
        elif direction == "D":
            return Point(point.x+1, point.y)
        elif direction == "R":
            return Point(point.x, point.y+1)
        elif direction == "L":
            return Point(point.x, point.y-1)

    def get_debug_board (data, point, direction):
        current_position = np.zeros_like(data)
        
        if direction == "U":
            direction_symbol = "^"
        elif direction == "D":
            direction_symbol = "v"
        elif direction == "R":
            direction_symbol = ">"
        elif direction == "L":
            direction_symbol = "<"
        current_position[point] = direction_symbol
        return current_position

    light_cache = defaultdict(list)

    def light_ray(point, direction, data, record):
        next_point = point
        current_direction = direction
        
            
        while True:
            symbol = data[next_point]
            record[next_point] = "#"
            if (light_cache[next_point]) and (current_direction in light_cache[next_point]):
                break
            else:
                light_cache[next_point] = light_cache[next_point] + [current_direction]

            if verbose:
                debug_board = get_debug_board(data, next_point, current_direction)
                print(debug_board)
            if symbol == ".":
                next_point = step_in_direction(next_point, current_direction)
                if not is_in_field(next_point):
                    break
            elif symbol == "/":
                if current_direction == "U":
                    current_direction = "R"
                elif current_direction == "D":
                    current_direction = "L"
                elif current_direction == "R":
                    current_direction = "U"
                elif current_direction == "L":
                    current_direction = "D"
                
                next_point = step_in_direction(next_point, current_direction)

            elif symbol == "\\":
                if current_direction == "U":
                    current_direction = "L"
                elif current_direction == "D":
                    current_direction = "R"
                elif current_direction == "R":
                    current_direction = "D"
                elif current_direction == "L":
                    current_direction = "U"
                
                next_point = step_in_direction(next_point, current_direction)
            

            elif ((symbol == "|") and (current_direction in ["U", "D"])) or ((symbol == "-") and (current_direction in ["L", "R"])) :
                next_point = step_in_direction(next_point, current_direction)

            elif (symbol == "|") and (current_direction in ["L", "R"]):
                current_direction = "U"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, data, record)
                current_direction = "D"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, data, record)
            elif (symbol == "-") and (current_direction in ["U", "D"]):
                current_direction = "L"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, data, record)
                current_direction = "R"
                next_point = step_in_direction(next_point, current_direction)
                if is_in_field(next_point):
                    light_ray(next_point, current_direction, data, record)
            if not is_in_field(next_point):
                break
            else:
                if verbose:
                    print(next_point, current_direction)


    energized_tiles = []
    top_starting_point = [(Point(0, y), "D") for y in range(data.shape[1])]
    bottom_starting_point = [(Point(data.shape[0]-1, y), "U") for y in range(data.shape[1])]
    left_starting_point = [(Point(x, 0), "R") for x in range(data.shape[0])]
    right_starting_point = [(Point(x, data.shape[1]-1), "L") for x in range(data.shape[0])]
    all_starting_positions = top_starting_point+bottom_starting_point+left_starting_point+right_starting_point
    for starting_point, direction in all_starting_positions:
        light_cache = defaultdict(list)
        record = np.zeros_like(data)
        light_ray(starting_point, direction, data, record)
        energized_tiles.append(np.count_nonzero(record))

    return energized_tiles


sol = solution(data)
# sol = solution(test_data, verbose=False)

print(sol)
print(max(sol))
# display(sum(sol))


[6875, 21, 63, 6836, 6840, 6854, 51, 41, 47, 83, 78, 43, 6814, 26, 32, 6830, 43, 6814, 12, 6875, 31, 58, 6820, 95, 6818, 54, 43, 43, 6960, 12, 7172, 6908, 6864, 52, 65, 55, 6817, 6814, 84, 52, 42, 6814, 6814, 6814, 47, 6852, 6815, 6818, 6814, 6814, 6814, 6814, 6814, 6824, 6814, 6814, 6832, 6818, 6826, 6814, 6978, 6817, 6818, 47, 7059, 6836, 68, 6814, 6814, 6818, 49, 6814, 6832, 6831, 6998, 49, 94, 86, 7030, 6941, 6914, 6819, 6885, 6819, 6831, 6957, 32, 6819, 5, 6872, 5, 6814, 6816, 6814, 17, 40, 7040, 16, 6976, 61, 6837, 47, 19, 6906, 23, 6906, 6824, 6901, 23, 17, 23, 41, 6921, 49, 21, 77, 6847, 6868, 6814, 27, 7163, 6814, 50, 58, 94, 64, 22, 68, 7, 6814, 82, 8, 7, 43, 8, 42, 40, 7041, 6832, 6814, 6830, 6864, 49, 7041, 7041, 6857, 6844, 33, 6855, 6936, 6846, 7141, 6814, 149, 6841, 6814, 49, 7141, 7135, 33, 6889, 6840, 7148, 6900, 6856, 7194, 6850, 6826, 6814, 6856, 6833, 7093, 6996, 7069, 6817, 162, 6850, 6837, 6838, 7041, 6882, 6963, 165, 6829, 7041, 6839, 17, 6851, 6871, 6947, 6969, 